In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import librosa
import pickle
import tensorflow as tf
import os

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
path = './gdrive/My Drive/hw4/'
path_model = path + 'model/'

import pickle
with open(path+'hw4_tr7.pkl', 'rb') as f:
  loaded_train = np.array(pickle.load(f))
with open(path+'hw4_te7.pkl', 'rb') as f:
  loaded_test = np.array(pickle.load(f))  

In [0]:
np.random.shuffle(loaded_train)
loaded_train = loaded_train.reshape(-1,28,28,1)
loaded_test = loaded_test.reshape(-1,28,28,1)


In [0]:
Xinp = tf.placeholder('float',[None,28,28,1])

is_training = tf.placeholder(tf.bool)

from sklearn.model_selection import train_test_split


latent_dim =4
img_dim =28
lr =0.0001

In [6]:
initconv = tf.contrib.layers.variance_scaling_initializer(uniform=False,factor=2.0)

conv1 = tf.layers.conv2d(inputs=Xinp,filters=16,kernel_size=[5, 5],padding="same",activation=tf.nn.relu,kernel_initializer=initconv,bias_initializer=initconv)

conv2 = tf.layers.conv2d(inputs=conv1,filters=32,kernel_size=[3, 3],padding="same",activation=tf.nn.relu,kernel_initializer=initconv,bias_initializer=initconv)

pool1 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=[2,2])#,data_format = 'channels_first')

dropout = tf.layers.dropout(inputs=pool1, rate=0.2,training=is_training)

#conv3 = tf.layers.conv2d(inputs=dropout,filters=32,kernel_size=[3, 3],padding="same",activation=tf.nn.relu,kernel_initializer=initconv,bias_initializer=initconv)


pool1_flat = tf.layers.flatten(dropout)

dense = tf.layers.dense(inputs=pool1_flat, units=128, activation=tf.nn.relu,kernel_initializer=initconv,bias_initializer=initconv)#,data_format = 'channels_first')

z_mean = tf.layers.dense(inputs=dense, units=latent_dim, activation=None, name='z_mean')#kernel_initializer=init,bias_initializer=init)

z_var = tf.layers.dense(inputs=dense, units=latent_dim, activation=None, name='z_var')#kernel_initializer=init,bias_initializer=init)

eps = tf.random_normal(shape=tf.shape(z_var),mean=0, stddev=1, dtype=tf.float32)
z = z_mean + tf.sqrt(tf.exp(z_var)) * eps

kl_loss = tf.reduce_sum(-0.5 * tf.reduce_sum(1 + z_var - tf.square(z_mean) - tf.exp(z_var), axis=1))

#up = tf.keras.layers.UpSampling2D(2,interpolation="bilinear")(z)
#Decoder
dedense = tf.layers.dense(inputs=z, units=img_dim*img_dim, activation=tf.nn.relu,kernel_initializer=initconv,bias_initializer=initconv)#,data_format = 'channels_first')

recon_image = tf.reshape(dedense, [-1,img_dim,img_dim,1])

deconv1 = tf.layers.conv2d(inputs=recon_image,filters=32,kernel_size=[3, 3],padding="same",activation=tf.nn.relu,kernel_initializer=initconv,bias_initializer=initconv)

dedrop = tf.layers.dropout(inputs=deconv1, rate=0.2,training=is_training)

#deconv2 = tf.layers.conv2d(inputs=dedrop,filters=16,kernel_size=[3, 3],padding="same",activation=tf.nn.relu,kernel_initializer=initconv,bias_initializer=initconv)

deconv3 = tf.layers.conv2d(inputs=dedrop,filters=1,kernel_size=[3, 3],padding="same",activation=tf.nn.sigmoid,kernel_initializer=initconv,bias_initializer=initconv)

decoded_image = tf.reshape(deconv3, [-1,img_dim,img_dim,1])

mse = tf.losses.mean_squared_error(labels=Xinp, predictions = decoded_image)
mse_loss= mse*img_dim*img_dim

tot_loss = mse_loss+kl_loss

optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(tot_loss)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [0]:
sess = tf.Session()
saver = tf.train.Saver()
init = tf.global_variables_initializer()


In [8]:
sess.run(tf.global_variables_initializer())
num_epochs = 500
batch_size=20


#new_list = [data_list[i:i+3] for i in range(0, len(data_list), 3)]

for epoch in range(num_epochs):
  loss_tot = 0

  for i in range(0,int(len(loaded_train)/batch_size)):   
    xbatch = loaded_train[i*batch_size:(i+1)*batch_size]
    last= (i+1)*batch_size
    res = sess.run([optimizer,tot_loss],feed_dict={Xinp:xbatch,is_training:True})
    loss_tot += res[1]

  xbatch = loaded_train[last:]

  if len(xbatch) >1:
    res = sess.run([optimizer,tot_loss],feed_dict={Xinp:xbatch,is_training:True})
    loss_tot += res[1]
  print("epoch",epoch,loss_tot)         
saver.save(sess, path_model + 'model_vae.ckpt')


epoch 0 28182.604606628418
epoch 1 23374.018321990967
epoch 2 22986.01356124878
epoch 3 22660.423057556152
epoch 4 22307.097274780273
epoch 5 21933.10301208496
epoch 6 21554.53115081787
epoch 7 21131.56956100464
epoch 8 20665.768306732178
epoch 9 20157.775676727295
epoch 10 19575.04236984253
epoch 11 18973.609008789062
epoch 12 18447.19912338257
epoch 13 17965.642223358154
epoch 14 17556.18846130371
epoch 15 17235.882846832275
epoch 16 16940.795513153076
epoch 17 16701.242584228516
epoch 18 16528.293922424316
epoch 19 16351.492900848389
epoch 20 16162.014377593994
epoch 21 16036.087860107422
epoch 22 15971.339027404785
epoch 23 15858.070610046387
epoch 24 15797.682704925537
epoch 25 15680.505393981934
epoch 26 15636.734149932861
epoch 27 15551.83090209961
epoch 28 15482.667583465576
epoch 29 15426.530834197998
epoch 30 15386.918830871582
epoch 31 15358.225521087646
epoch 32 15304.726512908936
epoch 33 15237.519134521484
epoch 34 15216.96174621582
epoch 35 15189.685779571533
epoch 36 15

KeyboardInterrupt: ignored

In [0]:
ran_int = np.array([np.random.uniform() for i in range(latent_dim)]).reshape(-1,latent_dim)
 vprint(ran_int)

plt.figure(0,figsize=(20,10))
for i in range(latent_dim):
    for j in range(0,100):
      js = int(j/10)
      tmp = ran_int
      tmp[:,i]=0 
      tmp[:,i]+= js
      ax = plt.subplot2grid((4,10),(i,js))
      ax.imshow(np.reshape(sess.run([decoded_image],feed_dict={z:tmp,is_training:False}), [28, 28]), cmap='gray')
plt.show()

#res = sess.run([decoded_image],feed_dict={z:ran_int,is_training:False})
#print(res[0])